# Instructions on how to prepare the model used in the following publication:
B. Pickering, R. Choudhary. Mitigating risk in district-level energy investment decisions by scenario optimisation, In: Proceedings of the 4th IBPSA-England Conference BSO 2018, Emmanuel College, Cambridge, 2018

In [2]:
## Import relevant packages
import os

import xarray as xr
import pandas as pd
import numpy as np

import calliope
from calliope.core import time

## internal package which includes functions for scenario reduction
import utils

os.chdir('model')

## Mean model

In [ ]:
## Build the mean model, to have a model to which we can manually apply clustering
mean_model = calliope.Model('model.yaml')

## Get user-defined clusters (monthly weekday/weekend)
clusters = pd.read_csv(os.path.join('timeseries_data', 'BSO2018', 'clusters.csv'), 
                       header=None, index_col=0, parse_dates=True)[1]

## Update timeseries to fit clusters
timeseries_data = mean_model._model_data.copy()
timeseries_data = timeseries_data.drop([
    i for i in timeseries_data.variables
    if 'timesteps' not in timeseries_data[i].dims or 'timestep_' in i
])

for dim in timeseries_data.dims:
    timeseries_data[dim] = mean_model._model_data[dim]
    
timeseries_data.attrs['_daily_timesteps'] = [
    mean_model._model_data.timestep_resolution.loc[i].values
    for i in np.unique(mean_model._model_data.timesteps.to_index().strftime('%Y-%m-%d'))
][0]
new_model_data = time.clustering.map_clusters_to_data(timeseries_data, clusters, how='mean')
for v in new_model_data.data_vars.values():
    v.attrs['is_result'] = 0
new_model_data = time.funcs._copy_non_t_vars(mean_model._model_data, new_model_data)
data_coords = mean_model._model_data.copy().coords
del data_coords['timesteps']
new_model_data.update(data_coords)

mean_model._model_data = new_model_data
mean_model.inputs = mean_model._model_data.filter_by_attrs(is_result=0)
## Save to netcdf if desired, to avoid running this step again
# mean_model.to_netcdf('mean_model_clustered_timeseries.nc') 

## Individual scenario

In [ ]:
## Load a scenario, e.g. scenario 1, into the model

def get_scenario_model(scenario):
    scenario_demand = pd.DataFrame(index=mean_model.inputs.timesteps)
    for energy in ['cooling', 'electricity', 'UPS']:
        ## get scenario demand from compressed CSV (gzip compression applied to reduce repository size)
        _demand = pd.read_csv(
            os.path.join('timeseries_data', 'scenarios', 'demand_{}_{}.csv.gz'.format(energy, scenario)),
            header=0, index_col=0, parse_dates=True
        )
        ## location -> loc::tech terminology, as used in Calliope
        _demand.rename(columns={i: '{}::demand_{}'.format(i, energy) for i in _demand.columns}, inplace=True)
        scenario_demand = scenario_demand.join(_demand)
    index_slice = {'loc_techs_finite_resource': scenario_demand.columns, 'timesteps': scenario_demand.index.values}

    ## Update relevant demand data in the model. We create a new Calliope model here, so the mean model remains intact
    scenario_model = calliope.Model(config=None, model_data=mean_model._model_data.copy(deep=True))
    scenario_model._model_data.resource.loc[index_slice] = scenario_demand.T.values

    return scenario_model

scenario_model = get_scenario_model(scenario=1)

## Scenario reduction

In [ ]:
## To undertake scenario reduction, run all scenarios independently, then load them into one xarray Dataset:
all_scenarios = xr.concat(
    [xr.open_dataset('path_to_scenario_model_{}'.format(i)) for i in range(1, 501)], 
    dim=pd.Index(data=[i for i in range(1, 501)], name='scenarios'),
    data_vars='different'
)

## Then run scenario reduction
reduced_scenarios = utils.get_reduced_scenarios(all_scenarios.cost.values, 16)
reduced_scenarios_df = utils.get_redistributed_probabilities(all_scenarios.cost.values, reduced_scenarios)

## Scenario optimisation

In [ ]:
## Create multi-scenario model, for scenario optimisation
scenarios = pd.read_csv(os.path.join('timeseries_data', 'reduced_scenarios.csv'), header=0, index_col=0)
scenario_model = calliope.Model(
    config=None, 
    model_data=xr.concat(
        [get_scenario_model(int(i))._model_data for i in scenarios.reduced_scenario.unique()], 
        dim=pd.Index(data=scenarios.reduced_scenario.astype(int).unique(), name='scenarios'), data_vars='different'
    )
)

## Add scenario probability
scenario_model._model_data['probability'] = xr.DataArray.from_dict({
    'data': scenarios.loc[scenario_model._model_data.scenarios.values, 'probability'].values,
    'dims': ('scenarios')
})
scenario_model._model_data['probability'].attrs['is_result'] = 0

## Add SO-related attributes
SO_attrs = {
    'run.mode': 'robust_plan',
    'run.beta': 5,
    'run.alpha': 0.9,
    'run.objective': 'robust_optimal_cost_minimization'
}
scenario_model._model_data.attrs.update(SO_attrs)

## Edit solver, if necessary
# scenario_model._model_data.attrs['run.solver'] = ''

## Recommended to save this to file, for running on a remote cluster:
# scenario_model.to_netcdf('scenario_model.nc')

## If you want to run right here:
# scenario_model.run()

## Unmet demand test

In [ ]:
# Update bigM used for unmet demand
scenario_model._model_data.attrs['run.bigM'] = 1e3 